In [ ]:
# uncomment and run the lines below if running in google colab
!pip install tensorflow==2.4.3
!git clone https://github.com/jlaihong/image-super-resolution.git
!mv image-super-resolution/* ./

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.6/394.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 86.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 77.8 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=423a00b13f4341578f7c35642307e127f9cfe0324d465b72fd6cfe4a3ab15f6a
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import glob
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf

from datasets.div2k.parameters import Div2kParameters 
from models.srresnet import build_srresnet
from models.pretrained import pretrained_models
from utils.prediction import get_sr_image
from utils.config import config

In [ ]:
dataset_key = "bicubic_x4"

data_path = config.get("data_path", "") 

div2k_folder = os.path.abspath(os.path.join(data_path, "div2k"))

dataset_parameters = Div2kParameters(dataset_key, save_data_directory=div2k_folder)

In [ ]:
def load_image(path):
    img = Image.open(path)
    
    was_grayscale = len(img.getbands()) == 1
    
    if was_grayscale or len(img.getbands()) == 4:
        img = img.convert('RGB')
    
    return was_grayscale, np.array(img)


In [ ]:
#model_name = "srresnet"
model_name = "srgan"

In [ ]:
model_key = f"{model_name}_{dataset_key}"

In [ ]:
weights_directory = os.path.abspath(f"weights/{model_key}")

file_path = os.path.join(weights_directory, "generator.h5")

if not os.path.exists(file_path):
    os.makedirs(weights_directory, exist_ok=True)
    
    print("Couldn't find file: ", file_path, ", attempting to download a pretrained model")
    
    if model_key not in pretrained_models:
        print(f"Couldn't find pretrained model with key: {model_key}, available pretrained models: {pretrained_models.key()}")
    else:
        download_url = pretrained_models[model_key]
        file = file_path.split("/")[-1]
        tf.keras.utils.get_file(file, download_url, cache_subdir=weights_directory)

Couldn't find file:  /content/weights/srgan_bicubic_x4/generator.h5 , attempting to download a pretrained model
6529024/6521184 [==============================] - 1s 0us/step


In [ ]:
model = build_srresnet(scale=dataset_parameters.scale)

os.makedirs(weights_directory, exist_ok=True)
weights_file = f'{weights_directory}/generator.h5'

model.load_weights(weights_file)

In [ ]:
image_path = "/content/drive/MyDrive/Datasets/test_report/baboon_LR.png"
was_grayscale, lr = load_image(image_path)

sr = get_sr_image(model, lr)
    
if was_grayscale:
    sr = ImageOps.grayscale(sr)

image_name = image_path.split("/")[-1]
sr.save("/content/drive/MyDrive/Datasets/test_report/baboon_SRGAN.png" )

In [ ]:
image_path = "/content/drive/MyDrive/Datasets/test_report/cartoon_LR.png"
was_grayscale, lr = load_image(image_path)

sr = get_sr_image(model, lr)
    
if was_grayscale:
    sr = ImageOps.grayscale(sr)

image_name = image_path.split("/")[-1]
sr.save("/content/drive/MyDrive/Datasets/test_report/cartoon_SRGAN.png" )

In [ ]:
#os.mkdir("/content/drive/MyDrive/Datasets/output")

In [ ]:
# lr_img_path = "/content/drive/MyDrive/Datasets/test_LR/X4"
# os.mkdir("/content/drive/MyDrive/Datasets/output/SRGAN_X4")
# for i in range(len(os.listdir(lr_img_path))):
#   image = os.listdir(lr_img_path)[i]
#   if i%100 == 0 : print(i)

#   image_paths = glob.glob(lr_img_path + "/" + image )

#   for image_path in image_paths:
#       #print(image_path)
#       was_grayscale, lr = load_image(image_path)
      
#       sr = get_sr_image(model, lr)
          
#       if was_grayscale:
#           sr = ImageOps.grayscale(sr)
      
#       image_name = image_path.split("/")[-1]
#       sr.save("/content/drive/MyDrive/Datasets/output/SRGAN_X4/"+image_name )

In [ ]:
# lr_img_path = "/content/drive/MyDrive/Datasets/test_LR/X3"
# os.mkdir("/content/drive/MyDrive/Datasets/output/SRGAN_X3")
# for i in range(len(os.listdir(lr_img_path))):
#   image = os.listdir(lr_img_path)[i]
#   if i%100 == 0 : print(i)

#   image_paths = glob.glob(lr_img_path + "/" + image )

#   for image_path in image_paths:
#       #print(image_path)
#       was_grayscale, lr = load_image(image_path)
      
#       sr = get_sr_image(model, lr)
          
#       if was_grayscale:
#           sr = ImageOps.grayscale(sr)
      
#       image_name = image_path.split("/")[-1]
#       sr.save("/content/drive/MyDrive/Datasets/output/SRGAN_X3/"+image_name )

In [ ]:
# lr_img_path = "/content/drive/MyDrive/Datasets/test_LR/X2"
# os.mkdir("/content/drive/MyDrive/Datasets/output/SRGAN_X2")
# for i in range(len(os.listdir(lr_img_path))):
#   image = os.listdir(lr_img_path)[i]
#   if i%100 == 0 : print(i)

#   image_paths = glob.glob(lr_img_path + "/" + image )

#   for image_path in image_paths:
#       #print(image_path)
#       was_grayscale, lr = load_image(image_path)
      
#       sr = get_sr_image(model, lr)
          
#       if was_grayscale:
#           sr = ImageOps.grayscale(sr)
      
#       image_name = image_path.split("/")[-1]
#       sr.save("/content/drive/MyDrive/Datasets/output/SRGAN_X2/"+image_name )